In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = [15, 4]
plt.style.use("fivethirtyeight")

pd.options.display.max_rows=1000

import warnings
warnings.simplefilter('ignore')

In [2]:
def get_data():
    train = pd.read_csv("train.csv")
    test = pd.read_csv("test.csv")
#     sub = pd.read_excel('sample_submission.xlsx')

    print(train.shape, test.shape)
    
    return train, test

In [3]:
train, test = get_data()


train['Loan_Amount_Requested'] = train['Loan_Amount_Requested'].str.replace(',','').astype(int)
train['Length_Employed'] = train['Length_Employed'].str.replace('<','')
train['Length_Employed'] = train['Length_Employed'].str.replace('+','')
train['Length_Employed'] = train['Length_Employed'].str.replace('year','')
train['Length_Employed'] = train['Length_Employed'].str.replace('s','')
# train['Closed_Accounts'] = train['Total_Accounts'] - train['Number_Open_Accounts']
# train['loan_div_income'] = train['Loan_Amount_Requested'] / train['Annual_Income']
# train['income_div_emplength'] = train['Annual_Income'] / train['Length_Employed']
# train['income_mul_emplength'] = train['Annual_Income'] * train['Length_Employed']
# train.drop(['Months_Since_Deliquency'],axis=1,inplace=True)
# train.drop([])



test['Loan_Amount_Requested'] = test['Loan_Amount_Requested'].str.replace(',','').astype(int)
test['Length_Employed'] = test['Length_Employed'].str.replace('<','')
test['Length_Employed'] = test['Length_Employed'].str.replace('+','')
test['Length_Employed'] = test['Length_Employed'].str.replace('year','')
test['Length_Employed'] = test['Length_Employed'].str.replace('s','')
# test['Closed_Accounts'] = test['Total_Accounts'] - test['Number_Open_Accounts']
# test['loan_div_income'] = test['Loan_Amount_Requested'] / test['Annual_Income']
# test['income_div_emplength'] = test['Annual_Income'] / test['Length_Employed']
# test['income_mul_emplength'] = test['Annual_Income'] * test['Length_Employed']
# test.drop(['Months_Since_Deliquency'],axis=1,inplace=True)


data = pd.concat([train, test], axis=0)
data['source'] = np.nan
data['source'].iloc[:train.shape[0]] = "train"
data['source'].iloc[train.shape[0]: ] = "test"
data.drop(['Interest_Rate'], axis=1, inplace=True)

print(data.shape)

(164309, 14) (109541, 13)
(273850, 14)


## Modelling 

In [4]:
train.isnull().sum()

Loan_ID                        0
Loan_Amount_Requested          0
Length_Employed             7371
Home_Owner                 25349
Annual_Income              25102
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    88379
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
Interest_Rate                  0
dtype: int64

In [5]:
test.isnull().sum()

Loan_ID                        0
Loan_Amount_Requested          0
Length_Employed             4936
Home_Owner                 16711
Annual_Income              16898
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    58859
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
dtype: int64

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164309 entries, 0 to 164308
Data columns (total 14 columns):
Loan_ID                    164309 non-null int64
Loan_Amount_Requested      164309 non-null int32
Length_Employed            156938 non-null object
Home_Owner                 138960 non-null object
Annual_Income              139207 non-null float64
Income_Verified            164309 non-null object
Purpose_Of_Loan            164309 non-null object
Debt_To_Income             164309 non-null float64
Inquiries_Last_6Mo         164309 non-null int64
Months_Since_Deliquency    75930 non-null float64
Number_Open_Accounts       164309 non-null int64
Total_Accounts             164309 non-null int64
Gender                     164309 non-null object
Interest_Rate              164309 non-null int64
dtypes: float64(3), int32(1), int64(5), object(5)
memory usage: 16.9+ MB


In [7]:
from pycaret.classification import *

In [8]:
exp_clf101 = setup(data = train, target = 'Interest_Rate', session_id=123)

 
Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Target Type,Multiclass
2,Label Encoded,None
3,Original Data,"(164309, 14)"
4,Missing Values,True
5,Numeric Features,7
6,Categorical Features,6
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [9]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,CatBoost Classifier,0.542,0,0.4952,0.5453,0.5305,0.2566
1,Light Gradient Boosting Machine,0.5399,0,0.4904,0.5443,0.5266,0.2513
2,Gradient Boosting Classifier,0.5312,0,0.4712,0.542,0.5094,0.2289
3,Extreme Gradient Boosting,0.5306,0,0.469,0.5418,0.5073,0.2268
4,Ada Boost Classifier,0.5296,0,0.4746,0.5342,0.5116,0.2306
5,Linear Discriminant Analysis,0.5152,0,0.444,0.5218,0.4799,0.1952
6,Ridge Classifier,0.5109,0,0.4305,0.5113,0.4606,0.1821
7,Random Forest Classifier,0.4903,0,0.4489,0.4837,0.4783,0.1798
8,Extra Trees Classifier,0.4874,0,0.4448,0.4803,0.477,0.174
9,Naive Bayes,0.4696,0,0.3883,0.4622,0.4129,0.1029


## Catboost

In [12]:
# create model
catboost = create_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.5368,0.0,0.4883,0.5373,0.5243,0.2479
1,0.5423,0.0,0.4963,0.5443,0.5316,0.2580
2,0.5453,0.0,0.4965,0.5492,0.5324,0.2609
3,0.5451,0.0,0.4982,0.5457,0.5334,0.2629
4,0.5399,0.0,0.4944,0.5468,0.5290,0.2525
5,0.5479,0.0,0.5019,0.5515,0.5370,0.2668
6,0.5416,0.0,0.4950,0.5456,0.5300,0.2559
7,0.5404,0.0,0.4920,0.5435,0.5282,0.2531
8,0.5454,0.0,0.4985,0.5478,0.5343,0.2627
9,0.5349,0.0,0.4907,0.5415,0.5245,0.2450


In [10]:
# tune model
tuned_cat = tune_model('catboost',optimize='F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.5359,0.0,0.4879,0.5356,0.5237,0.2471
1,0.5423,0.0,0.4957,0.5435,0.5314,0.2581
2,0.5434,0.0,0.4957,0.5463,0.5312,0.2586
3,0.5459,0.0,0.5011,0.5465,0.5352,0.2655
4,0.5428,0.0,0.4990,0.5479,0.5329,0.2589
5,0.5484,0.0,0.5036,0.5501,0.5382,0.2692
6,0.5381,0.0,0.4940,0.5400,0.5278,0.2523
7,0.5374,0.0,0.4910,0.5391,0.5263,0.2499
8,0.5402,0.0,0.4949,0.5432,0.5297,0.2547
9,0.5344,0.0,0.4915,0.5394,0.5247,0.2455


In [11]:
# predict model
data = predict_model(tuned_cat)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,One Vs Rest Classifier,0.5429,0,0.4965,0.5466,0.5316,0.2582


In [12]:
data.head()

,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Length_Employed_ 1,Length_Employed_1,Length_Employed_10,Length_Employed_2,...,Inquiries_Last_6Mo_2,Inquiries_Last_6Mo_3,Inquiries_Last_6Mo_4,Inquiries_Last_6Mo_5,Inquiries_Last_6Mo_6,Inquiries_Last_6Mo_7,Inquiries_Last_6Mo_8,Gender_Female,Interest_Rate,Label
0,14000,36000.000000,31.93,17.000000,12.0,23.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
1,4000,81500.000000,23.65,14.000000,9.0,32.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3
2,9000,73331.159434,13.99,14.000000,13.0,25.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,24250,103000.000000,12.56,34.229356,13.0,18.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
4,18000,112500.000000,9.09,34.229356,20.0,63.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1


In [13]:
tuned_cat.get_params()

{'estimator__iterations': 500,
 'estimator__learning_rate': 0.3,
 'estimator__depth': 4,
 'estimator__l2_leaf_reg': 10,
 'estimator__border_count': 200,
 'estimator__silent': True,
 'estimator__random_state': 123,
 'estimator': <catboost.core.CatBoostClassifier at 0x22bad336348>,
 'n_jobs': None}

In [14]:
# finalize model
final_cat = finalize_model(tuned_cat)

In [15]:
final_cat.get_params()

{'estimator__iterations': 500,
 'estimator__learning_rate': 0.3,
 'estimator__depth': 4,
 'estimator__l2_leaf_reg': 10,
 'estimator__border_count': 200,
 'estimator__silent': True,
 'estimator__random_state': 123,
 'estimator': <catboost.core.CatBoostClassifier at 0x22bad317948>,
 'n_jobs': None}

In [16]:
# predicting on final model
predict_model(final_cat)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,One Vs Rest Classifier,0.5738,0,0.5278,0.5807,0.5635,0.3088


,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Length_Employed_ 1,Length_Employed_1,Length_Employed_10,Length_Employed_2,...,Inquiries_Last_6Mo_2,Inquiries_Last_6Mo_3,Inquiries_Last_6Mo_4,Inquiries_Last_6Mo_5,Inquiries_Last_6Mo_6,Inquiries_Last_6Mo_7,Inquiries_Last_6Mo_8,Gender_Female,Interest_Rate,Label
0,14000,36000.000000,31.93,17.000000,12.0,23.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
1,4000,81500.000000,23.65,14.000000,9.0,32.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3
2,9000,73331.159434,13.99,14.000000,13.0,25.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,24250,103000.000000,12.56,34.229356,13.0,18.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
4,18000,112500.000000,9.09,34.229356,20.0,63.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49288,5600,105000.000000,6.66,43.000000,5.0,32.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49289,6000,60000.000000,17.14,66.000000,11.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49290,16000,61000.000000,9.21,34.229356,13.0,20.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49291,12000,73331.159434,23.49,31.000000,15.0,46.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1


In [17]:
# predict on test data
unseen_predictions = predict_model(final_cat, data=test)
unseen_predictions.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Label
0,10164310,27500,10,Mortgage,129000.0,VERIFIED - income,debt_consolidation,12.87,0,68.0,10,37,Male,2
1,10164311,26000,10,NaN,110000.0,not verified,credit_card,11.37,0,NaN,6,23,Male,1
2,10164312,6075,1,Rent,75000.0,VERIFIED - income,debt_consolidation,6.83,2,NaN,5,20,Male,2
3,10164313,12000,10,Mortgage,73000.0,VERIFIED - income source,debt_consolidation,7.76,0,NaN,6,8,Male,2
4,10164314,35000,1,Mortgage,156000.0,not verified,debt_consolidation,9.62,0,26.0,9,21,Male,2


In [18]:
sub = unseen_predictions[['Loan_ID','Label']]

In [19]:
sub = sub.rename(columns={'Label':'Interest_Rate'})

In [20]:
sub.head()

,Loan_ID,Interest_Rate
0,10164310,2
1,10164311,1
2,10164312,2
3,10164313,2
4,10164314,2


In [21]:
# this is the final submission 
sub.to_csv('final_catboost_submission.csv',index=None)

In [22]:
save_model(final_cat,'Final_catboost_submission_31May2020')

Transformation Pipeline and Model Succesfully Saved


### Light GBM

In [45]:
lgbm = tune_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.5360,0.0,0.4861,0.5356,0.5226,0.2461
1,0.5373,0.0,0.4910,0.5398,0.5260,0.2493
2,0.5387,0.0,0.4902,0.5418,0.5255,0.2503
3,0.5457,0.0,0.4979,0.5471,0.5331,0.2632
4,0.5389,0.0,0.4931,0.5452,0.5275,0.2510
5,0.5503,0.0,0.5025,0.5529,0.5385,0.2702
6,0.5357,0.0,0.4884,0.5378,0.5234,0.2463
7,0.5367,0.0,0.4881,0.5397,0.5241,0.2469
8,0.5390,0.0,0.4919,0.5403,0.5274,0.2522
9,0.5313,0.0,0.4852,0.5370,0.5195,0.2385


In [46]:
predict_model(lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,One Vs Rest Classifier,0.5409,0,0.4924,0.545,0.5281,0.2537


,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Length_Employed_ 1,Length_Employed_1,Length_Employed_10,Length_Employed_2,...,Inquiries_Last_6Mo_2,Inquiries_Last_6Mo_3,Inquiries_Last_6Mo_4,Inquiries_Last_6Mo_5,Inquiries_Last_6Mo_6,Inquiries_Last_6Mo_7,Inquiries_Last_6Mo_8,Gender_Female,Interest_Rate,Label
0,14000,36000.000000,31.93,17.000000,12.0,23.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
1,4000,81500.000000,23.65,14.000000,9.0,32.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3
2,9000,73331.159434,13.99,14.000000,13.0,25.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,24250,103000.000000,12.56,34.229356,13.0,18.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
4,18000,112500.000000,9.09,34.229356,20.0,63.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49288,5600,105000.000000,6.66,43.000000,5.0,32.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49289,6000,60000.000000,17.14,66.000000,11.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49290,16000,61000.000000,9.21,34.229356,13.0,20.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49291,12000,73331.159434,23.49,31.000000,15.0,46.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2


In [50]:
final_lgbm = finalize_model(lgbm)

In [51]:
predict_model(final_lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,One Vs Rest Classifier,0.5554,0,0.5081,0.561,0.5437,0.278


,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Length_Employed_ 1,Length_Employed_1,Length_Employed_10,Length_Employed_2,...,Inquiries_Last_6Mo_2,Inquiries_Last_6Mo_3,Inquiries_Last_6Mo_4,Inquiries_Last_6Mo_5,Inquiries_Last_6Mo_6,Inquiries_Last_6Mo_7,Inquiries_Last_6Mo_8,Gender_Female,Interest_Rate,Label
0,14000,36000.000000,31.93,17.000000,12.0,23.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
1,4000,81500.000000,23.65,14.000000,9.0,32.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3
2,9000,73331.159434,13.99,14.000000,13.0,25.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,24250,103000.000000,12.56,34.229356,13.0,18.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
4,18000,112500.000000,9.09,34.229356,20.0,63.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49288,5600,105000.000000,6.66,43.000000,5.0,32.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49289,6000,60000.000000,17.14,66.000000,11.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49290,16000,61000.000000,9.21,34.229356,13.0,20.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49291,12000,73331.159434,23.49,31.000000,15.0,46.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2


## Adaboost classifier

In [48]:
tuned_ada = tune_model('ada')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.5277,0.0,0.4690,0.5283,0.5070,0.2261
1,0.5268,0.0,0.4711,0.5323,0.5084,0.2250
2,0.5325,0.0,0.4746,0.5408,0.5120,0.2333
3,0.5362,0.0,0.4794,0.5408,0.5169,0.2410
4,0.5337,0.0,0.4764,0.5453,0.5140,0.2348
5,0.5338,0.0,0.4754,0.5413,0.5136,0.2350
6,0.5278,0.0,0.4710,0.5364,0.5081,0.2257
7,0.5288,0.0,0.4709,0.5323,0.5085,0.2277
8,0.5314,0.0,0.4730,0.5355,0.5112,0.2317
9,0.5253,0.0,0.4700,0.5357,0.5066,0.2218


In [49]:
predict_model(tuned_ada)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,One Vs Rest Classifier,0.5297,0,0.4723,0.5381,0.5098,0.2285


,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Length_Employed_ 1,Length_Employed_1,Length_Employed_10,Length_Employed_2,...,Inquiries_Last_6Mo_2,Inquiries_Last_6Mo_3,Inquiries_Last_6Mo_4,Inquiries_Last_6Mo_5,Inquiries_Last_6Mo_6,Inquiries_Last_6Mo_7,Inquiries_Last_6Mo_8,Gender_Female,Interest_Rate,Label
0,14000,36000.000000,31.93,17.000000,12.0,23.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
1,4000,81500.000000,23.65,14.000000,9.0,32.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3
2,9000,73331.159434,13.99,14.000000,13.0,25.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,24250,103000.000000,12.56,34.229356,13.0,18.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
4,18000,112500.000000,9.09,34.229356,20.0,63.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49288,5600,105000.000000,6.66,43.000000,5.0,32.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49289,6000,60000.000000,17.14,66.000000,11.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49290,16000,61000.000000,9.21,34.229356,13.0,20.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49291,12000,73331.159434,23.49,31.000000,15.0,46.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2


In [52]:
final_ada = finalize_model(tuned_ada)

In [53]:
predict_model(final_ada)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,One Vs Rest Classifier,0.5316,0,0.4748,0.5401,0.5123,0.232


,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Length_Employed_ 1,Length_Employed_1,Length_Employed_10,Length_Employed_2,...,Inquiries_Last_6Mo_2,Inquiries_Last_6Mo_3,Inquiries_Last_6Mo_4,Inquiries_Last_6Mo_5,Inquiries_Last_6Mo_6,Inquiries_Last_6Mo_7,Inquiries_Last_6Mo_8,Gender_Female,Interest_Rate,Label
0,14000,36000.000000,31.93,17.000000,12.0,23.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
1,4000,81500.000000,23.65,14.000000,9.0,32.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3
2,9000,73331.159434,13.99,14.000000,13.0,25.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,24250,103000.000000,12.56,34.229356,13.0,18.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
4,18000,112500.000000,9.09,34.229356,20.0,63.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49288,5600,105000.000000,6.66,43.000000,5.0,32.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49289,6000,60000.000000,17.14,66.000000,11.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49290,16000,61000.000000,9.21,34.229356,13.0,20.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49291,12000,73331.159434,23.49,31.000000,15.0,46.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2


In [54]:
stack_soft = stack_models([final_cat,final_lgbm], meta_model=final_cat)

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.5358,0.0,0.4868,0.5355,0.5230,0.2463
1,0.5399,0.0,0.4917,0.5414,0.5278,0.2530
2,0.5421,0.0,0.4923,0.5466,0.5284,0.2547
3,0.5418,0.0,0.4936,0.5424,0.5292,0.2566
4,0.5416,0.0,0.4945,0.5481,0.5297,0.2546
5,0.5464,0.0,0.4986,0.5492,0.5344,0.2636
6,0.5385,0.0,0.4905,0.5416,0.5260,0.2501
7,0.5378,0.0,0.4876,0.5414,0.5242,0.2475
8,0.5401,0.0,0.4929,0.5434,0.5285,0.2533
9,0.5326,0.0,0.4868,0.5380,0.5212,0.2407


In [55]:
predict_model(stack_soft)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Stacking Classifier,0.5438,0,0.4947,0.5488,0.5309,0.2578


,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Length_Employed_ 1,Length_Employed_1,Length_Employed_10,Length_Employed_2,...,Inquiries_Last_6Mo_2,Inquiries_Last_6Mo_3,Inquiries_Last_6Mo_4,Inquiries_Last_6Mo_5,Inquiries_Last_6Mo_6,Inquiries_Last_6Mo_7,Inquiries_Last_6Mo_8,Gender_Female,Interest_Rate,Label
0,14000,36000.000000,31.93,17.000000,12.0,23.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
1,4000,81500.000000,23.65,14.000000,9.0,32.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3
2,9000,73331.159434,13.99,14.000000,13.0,25.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,24250,103000.000000,12.56,34.229356,13.0,18.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
4,18000,112500.000000,9.09,34.229356,20.0,63.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49288,5600,105000.000000,6.66,43.000000,5.0,32.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49289,6000,60000.000000,17.14,66.000000,11.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49290,16000,61000.000000,9.21,34.229356,13.0,20.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
49291,12000,73331.159434,23.49,31.000000,15.0,46.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2


In [57]:
final_stack_soft = finalize_model(stack_soft)

IntProgress(value=0, description='Processing: ', max=17)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,19:55:36
Status,. . . . . . . . . . . . . . . . . .,Compiling Final Model
ETC,. . . . . . . . . . . . . . . . . .,0.0 Seconds Remaining


KeyboardInterrupt: 

In [58]:
# predict on test data
unseen_predictions = predict_model(stack_soft, data=test)
unseen_predictions.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Label
0,10164310,27500,10,Mortgage,129000.0,VERIFIED - income,debt_consolidation,12.87,0,68.0,10,37,Male,2
1,10164311,26000,10,NaN,110000.0,not verified,credit_card,11.37,0,NaN,6,23,Male,2
2,10164312,6075,1,Rent,75000.0,VERIFIED - income,debt_consolidation,6.83,2,NaN,5,20,Male,3
3,10164313,12000,10,Mortgage,73000.0,VERIFIED - income source,debt_consolidation,7.76,0,NaN,6,8,Male,2
4,10164314,35000,1,Mortgage,156000.0,not verified,debt_consolidation,9.62,0,26.0,9,21,Male,2


In [59]:
sub = unseen_predictions[['Loan_ID','Label']]

sub = sub.rename(columns={'Label':'Interest_Rate'})

sub.to_csv('stacking_soft_catboost_final.csv',index=None)